<center><h2><b>TRAIN MODELS</b></h2></center>

##### **CLEAN DATASET FOR FINETUNING GPT**

In [14]:
## FUNCTION CALL TO CLEAN DATASET
from utils.gpt_fine_tuning import run_movie_extraction_pipeline

# note that you need an OPENAI API KEY to run this function, as it uses CHATGPT to help with summaries

run_movie_extraction_pipeline(
    imdb_test_path="data/imdb_test.csv",
    labels_and_movies_path="data/imdb_test_labels_and_movies.csv",
    summaries_path="data/imdb_test_with_summaries.csv",
    cleaned_labels_and_movies_path="data/cleaned_labels_and_movies.csv",
    cleaned_reviews_path="data/cleaned_imdb_test.csv",
    cleaned_summaries_path="data/cleaned_imdb_test_with_summaries.csv",
    openai_api_key="sk-your-real-openai-key-here"
)


##### **INSTRUCTION FINETUNE GPT - takes about 6 hours on L4 GPU** 

In [ ]:
from utils.gpt_fine_tuning import finetune

## FUNCTION CALL TO TRAIN GPT

# use the cleaned dataset

train_file = "data/cleaned_imdb_train_25k.csv"
test_labels_reviews_file = "data/cleaned_imdb_test.csv"
test_labels_movies_file = "data/cleaned_labels_and_movies.csv"
test_summaries_file = "data/cleaned_imdb_test_with_summaries.csv"

# set model type (gpt2 in this case) and other calling parameters

model_name = "gpt2"
output_dir = "./models/gpt2-instruction-finetuned"
logging_dir = "./instruction-finetuning-logs"
num_train_epochs = 3
batch_size = 2

# if no plot is specified, we will just show() the plot
# you can set this to "./training_loss_plot.png" or some location to output a plot there. 

plot_save_file = None

# fine tune function

finetune(
    train_file=train_file,
    test_labels_reviews_file=test_labels_reviews_file,
    test_labels_movies_file=test_labels_movies_file,
    test_summaries_file=test_summaries_file,
    model_name=model_name,
    output_dir=output_dir,
    logging_dir=logging_dir,
    num_train_epochs=num_train_epochs,
    batch_size=batch_size,
    plot_save_file=plot_save_file
)

##### **PREPARE DATASET FOR FINETUNING LLAMA**

In [ ]:
from utils.llama_fine_tuning import create_augmented_imdb_dataset, add_prompt, add_augmented_prompt

## FUNCTION CALL TO CREATE AUGMENTED DATASET

output_dir = "data"
seed = 42
num_augmented = 10000

all_prompts_dataset = create_augmented_imdb_dataset(
    output_dir=output_dir,
    seed=seed,
    num_augmented=num_augmented
)

##### **FINETUNE LLAMA - takes about 24 hours on L4 GPU**

In [ ]:
from utils.llama_fine_tuning import finetune

## FUNCTION CALL TO TRAIN LLAMA

model_name = "meta-llama/Llama-2-7b-hf"
output_dir = "./model/llama-7b-qlora-finetuned"
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 4
learning_rate = 2e-4
num_train_epochs = 3

finetune(
    all_prompts_dataset=all_prompts_dataset,
    model_name=model_name,
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    num_train_epochs=num_train_epochs
)

----

<center><h2><b>EVALUATE PRETRAINED BERT</b></h2></center>

In [ ]:
import pandas as pd

train_df = pd.read_csv('data/imdb_train.csv')
test_df = pd.read_csv('data/imdb_test.csv')

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from utils.BERT_utils import classify_sentiment_bert_imdb_batch

# According to the official model card on Huggingface (https://huggingface.co/textattack/bert-base-uncased-imdb), 
# it was trained as follows (taken directly from the model card).

# This bert-base-uncased model was fine-tuned for sequence classification using TextAttack and the imdb dataset 
# loaded using the nlp library. The model was fine-tuned for 5 epochs with a batch size of 16, a learning rate 
# of 2e-05, and a maximum sequence length of 128. Since this was a classification task, the model was trained 
# with a cross-entropy loss function. The best score the model achieved on this task was 0.89088, as measured 
# by the eval set accuracy, found after 4 epochs.

# We will verify this model's performance on the training and testing set as shown below.

# use these lines to load a pre-trained version of BERT that was also fine-tuned on the IMDB dataset

bert_imdb_model_name = "textattack/bert-base-uncased-imdb"
tokenizer_bert_imdb = AutoTokenizer.from_pretrained(bert_imdb_model_name)
model_bert_imdb = AutoModelForSequenceClassification.from_pretrained(bert_imdb_model_name)
model_bert_imdb.eval()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_bert_imdb.to(device)

In [ ]:
# Evaluate the BERT model on the original training set

train_texts = train_df["text"].tolist()
train_labels = train_df["label"].tolist()

batch_size = 32
correct = 0
total = len(train_texts)

for i in range(0, total, batch_size):
    last_index = min(i + batch_size, total)
    batch_texts = train_texts[i:last_index]
    batch_labels = train_labels[i:last_index]
    
    pred_labels, _ = classify_sentiment_bert_imdb_batch(model_bert_imdb, tokenizer_bert_imdb, batch_texts, device=device)

    correct += sum(p == t for p, t in zip(pred_labels, batch_labels))

print(f"BERT Sentiment Accuracy on Train Set: {correct/total:.2%}")

In [ ]:
# Evaluate the BERT model on the original testing set

test_texts = test_df["text"].tolist()
test_labels = test_df["label"].tolist()

batch_size = 32
correct = 0
total = len(test_texts)

for i in range(0, total, batch_size):
    last_index = min(i + batch_size, total)
    batch_texts = test_texts[i:last_index]
    batch_labels = test_labels[i:last_index]
    
    pred_labels, _ = classify_sentiment_bert_imdb_batch(model_bert_imdb, tokenizer_bert_imdb, batch_texts, device=device)
    #print(pred_labels, batch_labels)

    correct += sum(p == t for p, t in zip(pred_labels, batch_labels))

print(f"BERT Sentiment Accuracy on Test Set: {correct/total:.2%}")

According to the experiemnts above, we have verified that the pretrained BERT from textattack scores about ~89% on the testing set. Thus, we judge that this off-the-shelf model is good enough to perform sentiment classification on the decoder output without additional finetuning. 

----

<center><h2><b>LOAD CLEANED DATASETS</b></h2></center>

In [1]:
import pandas as pd
import csv

In [2]:
test_labels_reviews_df = pd.read_csv('data/cleaned_imdb_test.csv')
test_labels_reviews_df.rename(columns={'text': 'review'}, inplace=True)

test_labels_movienames_df = pd.read_csv('data/cleaned_labels_and_movies.csv')
test_labels_movienames_summary_df= pd.read_csv('data/cleaned_imdb_test_with_summaries.csv')

In [3]:
test_labels_reviews_df.sample(3)

,review,label
6286,The only thing I expected that this film didn'...,0
2796,Despite the solid performance of Penelope Ann ...,0
16075,This is one of the anime films from The Animat...,1


In [4]:
test_labels_movienames_summary_df.sample(3)

,label,movie_name,summary
4241,0,Skeleton Man,"A military team is hunted by a mysterious, sup..."
17209,1,Hitch,"A professional ""date doctor"" who helps men woo..."
19045,1,Angels of the Universe,"""Angels of the Universe"" follows the life of P..."


In [5]:
test_df = pd.concat([test_labels_movienames_summary_df, test_labels_reviews_df["review"]], axis=1)
test_df.sample(3)

,label,movie_name,summary,review
10044,1,Apocalypse Now,"During the Vietnam War, Captain Willard is sen...",You can find an anti-war statement here withou...
12880,1,The Whole Nine Yards.,A mild-mannered dentist's life is turned upsid...,"I can not say this movie was a hilarious, but ..."
45,0,Witchery,A group of people become trapped in an abandon...,Witchcraft/Witchery/La Casa 4/ and whatever el...


In [6]:
train_df = pd.read_csv(
    "data/cleaned_imdb_train_25k.csv",
    engine="python",
    quoting=csv.QUOTE_ALL,
    quotechar='"',
    escapechar='\\',
    on_bad_lines='warn'  
)

train_df.sample(3)

,label,movie_name,summary,review
1881,0,In Cold Blood,"""In Cold Blood"" is a 1967 film directed by Ric...",In Cold Blood was one of several 60s films tha...
1638,0,Envy,"""Envy"" is a 2004 comedy film starring Ben Stil...","Jack Black's character, Tim Dingman the ""Dream..."
11594,1,A Christmas Carol,"""A Christmas Carol"" is a classic tale by Charl...",I saw this movie in sixth grade around Christm...


In [8]:
# LOAD FUNCS TO FETCH MODEL AND RUN EVALUTAION PIPELINE

from utils.inference_eval import load_model, evaluate_model

----

<center><h2><b>EVALUATE PRETRAINED MODELS</b></h2></center>

#### **GPT-PRETRAINED**

In [10]:
# Load your test DataFrame (replace this with your actual loading logic)
test_df_pretrained = test_df.sample(200)

# Evaluate with pretrained GPT-2
gpt2_model_pretrained, gpt2_tokenizer_pretrained = load_model(model_type="gpt2", pretrained=True)
evaluate_model((gpt2_model_pretrained, gpt2_tokenizer_pretrained), test_df_pretrained, prompt_type="label-moviename")


100%|██████████| 13/13 [00:50<00:00,  3.87s/it]


--- First 30 Generated Examples ---
1. Movie: Fire on the Amazon
   Sentiment: negative
   Generated Review: The film is not an actual comic, it's an action film. So it's a story about a superhero. And you really get a sense of the movie, the story of the comic book.

I'm gonna say this about the book: "This is my favorite book of all time. It's about a hero who's a super-hero. And when you put him down in the first book, it's like, 'This is the first time I've ever read this.'" And I think that's something that you should always read, but I mean, it's a comic book. I'm not gonna make the same mistake as I did with a comic book. I'll say this, but it's not a comic book, it's a story about a superhero. And it's about the hero who's got all the answers.

And it's really good, and I think that

2. Movie: A Sound of Thunder
   Sentiment: negative
   Generated Review: It's not a bad movie. The characters are good and the plot is good. But it's not a good movie. I like it. It's a movie that

(0.0051, 0.1265, 0.2521, 0.59)

In [11]:
evaluate_model((gpt2_model_pretrained, gpt2_tokenizer_pretrained), test_df_pretrained, prompt_type="label-moviename-summary")

  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:35<00:00,  2.72s/it]


--- First 30 Generated Examples ---
1. Movie: Fire on the Amazon
   Sentiment: negative
   Generated Review: The film takes place in an Amazon forest in the Amazon rainforest, a part of the Amazon forest where the largest number of people live. In the film, a group of anthropologists investigate the disappearance of a young woman from the forest and find evidence of the corruption and environmental destruction that have plagued the Amazon rainforest for generations.

The film tells the story of a young woman named Asa who is missing and she has been unable to find her way home after being kidnapped by her father. She is abducted and kidnapped by an unknown man who, in the film, is using the Amazon

2. Movie: A Sound of Thunder
   Sentiment: negative
   Generated Review: The book is an adaptation of the book. The first three issues are the last two pages of the book.

This is the first movie in the series to be adapted from the original screenplay. The first three issues of the book we

(0.0039, 0.1202, 0.2078, 0.54)

### **LLAMA-PRETRAINED**

In [12]:
test_df_pretrained = test_df_pretrained.sample(30)

# Evaluate with pretrained GPT-2
llama_model_pretrained, llama_tokenizer_pretrained = load_model(model_type="llama", pretrained=True)
evaluate_model((llama_model_pretrained, llama_tokenizer_pretrained), test_df_pretrained, prompt_type="label-moviename")

100%|██████████| 2/2 [01:32<00:00, 46.36s/it]


--- First 30 Generated Examples ---
1. Movie: The Manson Family
   Sentiment: negative
   Generated Review: Unterscheidung zwischen Kern und Schale, die nach dem Schalensatz zu unterscheiden sind, liegt meist in der Bezeichnung des Schalensatzes.
Negative review.
Negative reviews are those that are not positive.
Negative reviews are those that are not positive. A negative review can be a review that is either negative in nature or not positive in nature. A negative review is a review that is not positive. A negative review can be a review that is either negative in nature or not positive in nature. A negative review can be a review that is either negative in nature or not positive in nature. A negative review can be a review that is either negative in nature or not positive in nature. A negative review can be a review that is either negative in nature or not positive in nature. A negative review can be a review that is either negative

2. Movie: Thirst
   Sentiment: positive
   Genera

(0.0025, 0.0892, 0.1249, 0.5333)

In [13]:
# Evaluate with pretrained GPT-2
evaluate_model((llama_model_pretrained, llama_tokenizer_pretrained), test_df_pretrained, prompt_type="label-moviename-summary")

100%|██████████| 2/2 [01:01<00:00, 30.92s/it]


--- First 30 Generated Examples ---
1. Movie: The Manson Family
   Sentiment: negative
   Generated Review: The movie is set to release in 2023.
The Manson Family movie is about a cult leader who manipulates his followers to commit murder.
The Manson Family movie is about a cult leader who manipulates his followers to commit murder. The movie is based on the true story of Charles Manson and his followers, who committed a series of murders in the 1960s. The movie explores the events leading up to the murders, as well as the aftermath of the crimes. The Manson Family is a crime drama

2. Movie: Thirst
   Sentiment: positive
   Generated Review: If you liked the movie, please write a positive review.
I don't know how long I'll be able to do this, so don't wait too long.
I'll be checking in to see if you've done it. If not, I'll do it for you.
Give me a positive review for the movie Thirst. This is a short summary of the movie: "Thirst" is a 2009 South Korean horror film about a priest wh

(0.003, 0.1082, 0.1756, 0.5667)

<center><h2><b>INFERENCING ON GPT</b></h2></center>

In [8]:
train_sample_df_gpt = train_df.sample(200)
test_sample_df_gpt = test_df.sample(200)

In [9]:
gpt_model, gpt_tokenizer = load_model('gpt2-summaries')

##### **DEFAULT TOP-K, TOP-P, TEMPERATURE**

<center>TEST SET</center>

In [10]:
evaluate_model(
        model_and_tokenizer = (gpt_model, gpt_tokenizer),
        df=test_sample_df_gpt,
        prompt_type='label-moviename',
        
    )

100%|██████████| 13/13 [00:48<00:00,  3.71s/it]


--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: The first half hour is OK, the second half decent, but the last half is a bit of a disappointment. I have no idea why this movie was made. It's not in any way good, or interesting. It's a bad movie, and bad in every way. The movie is just a cheap excuse to have a movie. I give it 2.5 stars, but not enough to redeem it for me. I don't know why it was on the list of worst movies I have seen in a long time. But I guess it's all a cheap excuse to make a movie that doesn't even try to be entertaining. And it's not even that good either.<br /><br />What was the point of this movie? The main characters? The main storyline? The main story? The main characters? I don't know. The movie is so bad it

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: I have watched it twice, both times in succession and on the first time I was watching. I wanted to think I coul

(0.0065, 0.1377, 0.6722, 0.98)

In [14]:
evaluate_model(
        model_and_tokenizer = (gpt_model, gpt_tokenizer),
        df=test_sample_df_gpt,
        prompt_type='label-moviename-summary',
    )

100%|██████████| 13/13 [00:35<00:00,  2.76s/it]


--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: This was the worst movie I have ever seen. The plot was terrible. I don't think the director was aware that he was making this movie. It looked like a film with a cast that was trying to be funny. I don't think it was meant to be funny. I think the movie was not meant to be a comedy. It was meant to be an action movie. The acting was horrible. It was not funny at all. It was not interesting. The scene where the girl is on the toilet is just so bad that I couldn't stand to watch it. The songs were horrible. It was so bad that I wanted to turn it off

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: This movie is a classic. I am a huge fan of the horror genre and have watched this movie many times. So many movies have been made where a movie fails in every way, and this one is no exception. If you have not seen it, I strongly suggest you do not. This

(0.0057, 0.1359, 0.6609, 0.935)

<center> TRAIN SET </center>

In [15]:
evaluate_model(
        model_and_tokenizer = (gpt_model, gpt_tokenizer),
        df=train_sample_df_gpt,
        prompt_type='label-moviename',
    )

100%|██████████| 13/13 [00:48<00:00,  3.72s/it]


--- First 30 Generated Examples ---
1. Movie: Forbidden Lies
   Sentiment: positive
   Generated Review: This is a pretty good movie, with a lot of nudity and gore. The story is a bit thin at times but the acting is pretty good. The movie has lots of nudity and blood. This is definitely not a bad movie, but I'm not going to go so far as to call it a bad movie. I've never seen a movie that is as bad as Forbidden Lies. I just found it a bit disturbing, but still worth watching. The movie is pretty good, and it has a very good ending. It is a good movie, but not without its flaws. I've seen bad movies before and this one is the first one I've seen that has it all. I like the gore and the acting. I've seen bad movies before but this one has it all. I like the gore and the acting. I've seen bad movies before and this one has it all

2. Movie: Anchors Aweigh
   Sentiment: positive
   Generated Review: The trailer trailer for this movie was on a big screen at the Toronto International Film F

(0.006, 0.1397, 0.7883, 0.97)

In [16]:
evaluate_model(
        model_and_tokenizer = (gpt_model, gpt_tokenizer),
        df=train_sample_df_gpt,
        prompt_type='label-moviename-summary',
    )

100%|██████████| 13/13 [00:16<00:00,  1.29s/it]


--- First 30 Generated Examples ---
1. Movie: Forbidden Lies
   Sentiment: positive
   Generated Review: This film is a must see for any journalist and any fan of the subject matter. The film is well acted and the story line is well presented. The documentary has a very interesting conclusion but it

2. Movie: Anchors Aweigh
   Sentiment: positive
   Generated Review: This is a movie that will entertain you for days. It is a simple story of two sailors on leave in Los Angeles. Joe (Gene Kelly) is a successful businessman who is in the market

3. Movie: Shall We Dance?
   Sentiment: positive
   Generated Review: This movie is a good example of how to get over a movie and have it go on forever. It has a great story, great acting, and the best plot in the world. The acting

4. Movie: "American Virgin"
   Sentiment: negative
   Generated Review: This movie was a major disappointment for me. I was so bored out of my mind watching this that I could barely sit through it. I did, however, lik

(0.0019, 0.1025, 0.6191, 0.855)

##### **VARYING P-VALUEs**

In [17]:
for prompt_type in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (gpt_model, gpt_tokenizer),
        df=test_sample_df_gpt,
        prompt_type=prompt_type,
        top_p=0.4,
    )

100%|██████████| 13/13 [00:48<00:00,  3.72s/it]



--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: I was expecting a good movie, but it was a disappointment. I was expecting a good movie, but it was a disappointment. I'm not sure what the reason was. I don't know what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason was. I'm not sure what the reason

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: I was expecting a good movie, but I was disappointed. I was expecting a good movie. I was expecting a good movie. I was

100%|██████████| 13/13 [00:36<00:00,  2.80s/it]


--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: This is a very short movie, but it is worth watching. I have seen a lot of short movies, but this one is the worst. I don't know what the director was thinking, but this movie is so bad that it is hard to believe that it was made by a director who is so famous. I don't know why the director didn't just make a short movie, but he made a movie about a singer who falls in love with a woman who is about to marry someone else. The movie is so bad that it is hard to believe that it was made by a director who is so famous. I don't know why the director didn

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: The film is a complete waste of time and money. I have to say that I was really looking forward to seeing this movie. I had to see it because I was really looking forward to seeing a movie about Halloween. I was really looking forward to seeing a movie 

In [18]:
for prompt_type in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (gpt_model, gpt_tokenizer),
        df=test_sample_df_gpt,
        prompt_type=prompt_type,
        top_p=0.7,
    )

100%|██████████| 13/13 [00:49<00:00,  3.84s/it]



--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: The acting is horrible. The script is horrible. The plot is a joke, the characters are just plain stupid. The acting is bad. The dialog is just bad. The movie is bad. It's so bad it's good. I don't even want to watch it. It's not even worth watching. I have to give it a 2 out of 10. The only thing that makes this movie even worse is the acting. I can't believe I wasted 2 hours of my life watching this. I know it's a comedy but it's not funny. The jokes are so stupid that they make you laugh. I mean, it's not even funny in the slightest. The acting is terrible. The script is bad. The acting is horrible. The story is just stupid. The movie is so bad that it's good. I have to give it a 2 out of 10

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: The movie is a waste of time. It's so bad that it's even worse than a bad TV movie. It's a waste of time. 

100%|██████████| 13/13 [00:39<00:00,  3.07s/it]


--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: I am a huge fan of the old, but now a new generation of singers has come to see this movie. I have heard that this movie is a waste of time. It's so boring and boring. I am a huge fan of the old, but now a new generation of singers has come to see this movie. I have heard that this movie is a waste of time. It's so boring and boring. I am a huge fan of the old, but now a new generation of singers has come to see this movie. I have heard that this movie is a waste of time. It's so boring and boring. I am a huge fan of

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: The kids are the only ones who can stand up to them, and they are the only ones who can save them.The acting is terrible. The characters are all horrible. The movie is boring. I would not recommend this movie to anyone. I would not recommend this movie to anyone who likes bad movies. I 

##### **VARYING K-VALUEs**

In [19]:
for prompt_type in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (gpt_model, gpt_tokenizer),
        df=test_sample_df_gpt,
        prompt_type=prompt_type,
        top_k=10,
    )

100%|██████████| 13/13 [00:49<00:00,  3.80s/it]



--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: The movie is so bad, so bad that it's almost laughable. I mean, I'm not even going to give a plot summary, but I'm not going to give a plot summary. The movie is so bad that it's even worse than a bad movie. This movie is a complete waste of time and money. If you want to watch this movie, you're going to have to go rent a lot of porno. The movie is so bad that I couldn't even get out of bed after watching it. The acting was awful. The story is so weak that you can't even think about it. The plot is so bad that it's laughable. The only thing that saves the movie is the fact that it's so bad that it's even worse than a bad movie. It's like watching a bad movie with a bunch of actors and they're trying to make

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: I'm a fan of the movie but the movie is not a good one at all. I am a big fan of the origina

100%|██████████| 13/13 [00:36<00:00,  2.80s/it]


--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: The movie was really a waste of money, but I was happy that it was a good movie. I was expecting to be disappointed. The movie was a waste of time. It was very slow and boring. I don't think it was meant to be a comedy, but it was. It was very slow and boring. I don't think it was meant to be a comedy, but it was. It was very slow and boring. I don't think it was meant to be a comedy, but it was. It was very slow and boring. I don't think it was meant to be a comedy, but it was. It was very slow and

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: It is an awful movie, with no suspense, bad acting, bad directing, and bad directing. The worst thing I can say is that I was watching this movie for the first time in ages, and I'm not even joking. The acting was terrible, the plot was bad, and the movie was horrible. The movie was supposed to be a come

In [20]:
for prompt_type in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (gpt_model, gpt_tokenizer),
        df=test_sample_df_gpt,
        prompt_type=prompt_type,
        top_k=300,
    )

100%|██████████| 13/13 [00:48<00:00,  3.74s/it]



--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: This is the only time I've seen a movie that was not as bad as I had hoped. I was watching it on a VHS tape of "The Sopranos" because the cable box showed an episode of it on "newscast". The only movie I liked was the episode of The Sopranos...I liked it! But I don't know if I'd ever get to see a good Sopranos...I think I'd never get to see it, and I really want to. I hope I'll find something that will make me watch it. The Sopranos was one of the best episodes of the Sopranos...It was really good, it had some good moments, but it was all over the place. The Sopranos was probably the worst episode of the Sopranos...I really wanted to get it, but I couldn't

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: The story was horrible. I don't think this is a coincidence. I loved it. This movie was really bad. I was expecting more. I was expecting somethi

100%|██████████| 13/13 [00:36<00:00,  2.78s/it]


--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: This movie is simply terrible. The songs are not funny or funny, the acting is so awful, the direction is poor and the direction is so bad it is not even funny.<br /><br />I don't know why I saw this movie. I saw the first one at a friend's house and she said "That's too bad. You can't make a movie about this". I can't believe I have to say this. This movie is so bad I can't even get out of my head. The acting is terrible, the story is so lame, and the direction is so bad it is funny. I can't believe that there was

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: The movie, which had a very well cast cast and the story line was very weak. It was all too predictable. This movie was pure crap. I gave it a 7 because I'm a horror movie fan, and I have seen a lot of movies. This is the worst movie I've ever seen. I think I'm going to see a whole lot mo

##### **VARYING TEMPERATURE**


In [21]:
for prompt_type in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (gpt_model, gpt_tokenizer),
        df=test_sample_df_gpt,
        prompt_type=prompt_type,
        temperature=0.1,
    )

100%|██████████| 13/13 [00:48<00:00,  3.69s/it]



--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: The movie is a complete waste of time. The acting is terrible, the story is bad, the story is bad, the story is bad. The only thing that makes this movie worth watching is the fact that it's a comedy. I'm not sure if it's a comedy or a drama, but it's not funny at all. I'm not sure if it's a comedy or a drama, but it's not funny at all. The only thing that makes this movie worth watching is the fact that it's a comedy. I'm not sure if it's a comedy or a drama, but it's not funny at all. The only thing that makes this movie worth watching is the fact that it's a comedy. I'm not sure if it's a comedy or a drama, but it's not funny at all. The only thing that makes this movie worth watching is

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: I'm not sure if it's because I'm a fan of the original series or just because I'm not a fan of the original se

100%|██████████| 13/13 [00:35<00:00,  2.76s/it]


--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: I was so disappointed with this movie. I was expecting a comedy, but it was just a bunch of songs and a bunch of songs. I was expecting a comedy, but it was just a bunch of songs and a bunch of songs. I was expecting a comedy, but it was just a bunch of songs and a bunch of songs. I was expecting a comedy, but it was just a bunch of songs and a bunch of songs. I was expecting a comedy, but it was just a bunch of songs and a bunch of songs. I was expecting a comedy, but it was just a bunch of songs and a bunch of songs. I was expecting a

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: The movie is so bad that it's funny. I'm not sure what the point of this movie is, but it's so bad that it's funny. I'm not sure what the point of this movie is, but it's so bad that it's funny. I'm not sure what the point of this movie is, but it's so bad that it's 

In [22]:
for prompt_type in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (gpt_model, gpt_tokenizer),
        df=test_sample_df_gpt,
        prompt_type=prompt_type,
        top_k=50,
        temperature=1.3,
    )

100%|██████████| 13/13 [00:47<00:00,  3.62s/it]



--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: The movie must have been filmed in about 2000 and I must write about those who did not know about this plot device. They would be able to identify the device in the opening scene with some type of video, in the end and the audience would have no idea.<br /><br />Firstly, let's start out by introducing to you that my favourite actress, Mira Shepovich(WOW) plays with her boyfriend on a movie set and also goes and goes all her own way into the film making herself for everybody on the inside. Also being an actress she's the single best in a porno scene(as is this reviewer)so even if her acting hasn't shown on screen the scenes here and there could certainly count the times they appeared(which the porn stars are not).<br /><br />I must say the plot is awful which means this is

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: For one, because it looked 

100%|██████████| 13/13 [00:36<00:00,  2.77s/it]


--- First 30 Generated Examples ---
1. Movie: Kahin Pyaar Na Ho Jaaye
   Sentiment: negative
   Generated Review: Oh my, I could write 1 review better! But this is NOT one of those who's supposed to care about an action movie (as some are saying.) I'll add it simply so it can serve as a guide for those wishing to try and read a review about movies not on this forum.<br /><br />A good actor goes on trial for having an accident with the death of his character, after they're asked by a judge to explain it to the jury who will decide their punishment in the near future. So while the jury finds the actor guilty, he's sentenced to probation until the end (where he never ever makes it to that point

2. Movie: Night of the Demons
   Sentiment: negative
   Generated Review: I actually thought I'd like this thing more and more. The problem is, the only problem with it is the characters - I'm a huge fan of Jack Lemmon, but to do such an atrocity as Night of the Demons in a cheap Halloween specia

<center><h2><b>INFERENCING ON LLAMA</b></h2></center>

In [23]:
train_sample_df = train_df.sample(30)
test_sample_df = test_df.sample(30)

In [24]:
llama_model, llama_tokenizer = load_model('llama')

Loading checkpoint shards: 100%|██████████| 2/2 [00:49<00:00, 24.60s/it]


##### **DEFAULT TOP-K, TOP-P, TEMPERATURE**

<center>TEST SET</center>

In [25]:
evaluate_model(
        model_and_tokenizer = (llama_model, llama_tokenizer),
        df=test_sample_df,
        prompt_type='label-moviename',
        batch_size = 2
    )

100%|██████████| 15/15 [11:10<00:00, 44.70s/it]


--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: Jackass: The Movie is a movie that is meant to be watched with friends. The movie is an assortment of crazy stunts and stupid situations that will make you laugh, or in my case, cry. The movie is made up of skits that show the guys doing things that are so ridiculous that they will make you laugh. The movie also has a storyline, which is that the guys are trying to break into the movie business. The storyline is just a way for the guys to come up with an excuse to do stunts. The movie is not for everyone, but if you have a good sense of humor, you will love it. I give Jackass: The Movie a 7/10. Overall, I recommend this movie to anyone who is a fan of the show or just looking for a good laugh. Jackass: The Movie is a

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: I'm not a fan of the "Southern Gothic" school of writing and I was afraid this movie wo

(0.009, 0.1349, 0.6573, 0.9667)

In [26]:
evaluate_model(
        model_and_tokenizer = (llama_model, llama_tokenizer),
        df=test_sample_df,
        prompt_type='label-moviename-summary',
        batch_size = 2
    )

100%|██████████| 15/15 [08:09<00:00, 32.60s/it]


--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: The stunts are sometimes dangerous, sometimes disgusting, and always funny. The movie is filled with hilarious and sometimes disgusting stunts. The movie is very funny, filled with hilarious stunts and pranks. I liked the movie very much. I give this movie a 10/10. I recommend this movie to everybody who likes to laugh. I am going to watch this movie again and again. I recommend this movie to everybody who likes to laugh. This is a good movie to watch with your friends. The stunts are sometimes dangerous, sometimes disgusting, and always

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: After watching this movie, I would be more than willing to bet that the director/producer/writer has never spent a moment in the Louisiana wetlands or anywhere in the south for that matter. The movie's plot is full of holes and the scenery is very unbelievable. It makes

(0.0068, 0.1343, 0.5551, 0.9667)

<center> TRAIN SET </center>

In [27]:
evaluate_model(
        model_and_tokenizer = (llama_model, llama_tokenizer),
        df=train_sample_df,
        prompt_type='label-moviename',
        batch_size = 2
    )

100%|██████████| 15/15 [11:08<00:00, 44.55s/it]


--- First 30 Generated Examples ---
1. Movie: Wonderland
   Sentiment: positive
   Generated Review: I must admit that I didn't really know anything about the Wonderland Murders before seeing this movie. What I do know is that it is based on a true story. The cast is great. The plot is a bit hard to follow in the beginning but once you get through the first hour, you'll find yourself hooked. The story is about a group of wannabe criminals who plan to rob a house and steal drugs. The robbery goes wrong and two members of the group end up dead and the rest are arrested. There are a lot of characters in this movie and the actors do a great job. The plot is somewhat confusing in the beginning but once you get through the first hour, you'll find yourself hooked. The movie is about 2 hours and 20 minutes long. I highly recommend this movie. I give it a 9/10.

2. Movie: The All Together
   Sentiment: negative
   Generated Review: This movie is a poorly written, poorly acted, poorly directed 

(0.0043, 0.1296, 0.5808, 0.9)

In [28]:
evaluate_model(
        model_and_tokenizer = (llama_model, llama_tokenizer),
        df=train_sample_df,
        prompt_type='label-moviename-summary',
        batch_size = 2
    )

100%|██████████| 15/15 [01:55<00:00,  7.72s/it]


--- First 30 Generated Examples ---
1. Movie: Wonderland
   Sentiment: positive
   Generated Review: <br /><br />The film is notable for its strong performances, particularly from Val Kilmer as John Holmes and Kate Bosworth as Dawn Schiller. The two actors bring a raw intensity to their roles, capturing the essence of their real-life counter

2. Movie: The All Together
   Sentiment: negative
   Generated Review: The performances by the cast are solid, especially by the two leads, Chris and Bob. The chemistry between them is well-developed, and the audience is drawn into their journey. The film has a strong cast, including James Corden, who delivers a hil

3. Movie: Grand Canyon
   Sentiment: positive
   Generated Review: The performances of the cast are remarkable, with Kevin Kline

4. Movie: Bloodsucka Jones vs. The Creeping Death
   Sentiment: positive
   Generated Review: The cast is led by the charismatic and charming

5. Movie: That's My Bush!
   Sentiment: positive
   Generated 

(0.0, 0.0597, 0.2325, 0.6333)

##### **VARYING P-VALUEs**

In [29]:
for prompt_type in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (llama_model, llama_tokenizer),
        df=test_sample_df,
        prompt_type=prompt_type,
        top_p=0.4,
        batch_size = 2
    )

100%|██████████| 15/15 [11:09<00:00, 44.65s/it]



--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: Jackass: The Movie is a movie that is not for the faint of heart. It is a movie that is filled with stupidity, but it is also filled with laughter. The movie is filled with stupidity because the cast members of the show are doing stupid things. The movie is filled with laughter because the cast members of the show are doing stupid things. The movie is filled with stupidity because the cast members of the show are doing stupid things. The movie is filled with laughter because the cast members of the show are doing stupid things. The movie is filled with stupidity because the cast members of the show are doing stupid things. The movie is filled with laughter because the cast members of the show are doing stupid things. The movie is filled with stupidity because the cast members of the show are doing stupid things. The movie is filled with laughter because the cast members of the 

100%|██████████| 15/15 [08:09<00:00, 32.61s/it]


--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: The movie is funny, but not as funny as the TV show. The movie is also very violent, and some of the stunts are very dangerous. The movie is also very long, and some of the stunts are very gross. The movie is also very stupid, and some of the stunts are very stupid. The movie is also very stupid, and some of the stunts are very stupid. The movie is also very stupid, and some of the stunts are very stupid. The movie is also very stupid, and some of the stunts are very stupid. The movie is also very stupid, and some of the stunts are

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: This is a long summary of the movie: A detective in post-Katrina Louisiana investigates a series of murders while experiencing visions of Confederate soldiers and grappling with his own personal demons. The movie is slow, the acting is mediocre, the story is not very interest

In [30]:
for prompt_types in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (llama_model, llama_tokenizer),
        df=test_sample_df,
        prompt_type=prompt_types,
        top_k=50,
        top_p=0.7,
        temperature=0.7,
        batch_size = 2
    )

100%|██████████| 15/15 [11:10<00:00, 44.67s/it]



--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: I'm a big fan of the show, and I think the movie was pretty good. It was funny, and it had some of the best stuff from the show. The movie had some new stuff, but I don't think it was as good as some of the stuff from the show. Overall, I thought it was a good movie, and I would recommend it to fans of the show. I would give it a 7 out of 10. I would recommend this movie to fans of the show. I would give it a 7 out of 10. I would recommend this movie to fans of the show. I would give it a 7 out of 10. I would recommend this movie to fans of the show. I would give it a 7 out of 10. I would recommend this movie to fans of the show. I would give it a

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: I am not a big fan of Tommy Lee Jones, but I thought he did a great job in this movie. I think the movie is a good mystery, but I don't think it is a great my

100%|██████████| 15/15 [08:08<00:00, 32.59s/it]


--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: The movie is hilarious and very entertaining. It is very funny, but it is also very crude and disgusting. The movie is a great movie to watch if you want to have a good laugh. The movie is very entertaining and very funny. The movie is very funny and very entertaining. The movie is very funny and very entertaining. The movie is very funny and very entertaining. The movie is very funny and very entertaining. The movie is very funny and very entertaining. The movie is very funny and very entertaining. The movie is very funny and very entertaining. The movie is

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: The story is set in the swamps of Louisiana. It's a slow-moving, atmospheric film, and the swamp setting is beautiful. The film is very slow-paced and there is little action. The movie is about a detective who is haunted by his past. He is not a par

##### **VARYING K-VALUEs**

In [31]:
for prompt_type in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (llama_model, llama_tokenizer),
        df=test_sample_df,
        prompt_type=prompt_type,
        top_k=10,
        top_p=0.95,
        temperature=0.7,
        batch_size = 2
    )

100%|██████████| 15/15 [11:09<00:00, 44.66s/it]



--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: What can I say? I'm a fan of the show, and I'm a fan of the first two movies. I can't say this is the greatest movie ever made, but it's funny. And if you're a fan of the show, this is for you. The movie is full of all of the usual Jackass antics, and I laughed quite a bit during the movie. The movie is full of new stunts, and there's a lot of old ones in there, too. It's a great mix of new and old, and it's great to see the cast back in action. The movie is full of new and old pranks, and it's great to see the cast back in action. I can't say this is the greatest movie ever made, but it's funny. And if you're a

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: What an awful movie! I have never seen anything so poorly directed and acted. I can't believe the actors in this movie were actually paid to work on this mess. This is the worst movie I have eve

100%|██████████| 15/15 [08:09<00:00, 32.60s/it]


--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: The film's success was attributed to the fact that it was the first Jackass movie. The movie was a huge success, making $133.7 million domestically and $206.1 million worldwide on a $5 million budget. The movie has a 5.4/10 on IMDB. It is a very funny movie. The movie is rated R for language, sex and nudity, violence, drug use, crude humor and brief sexuality. I would recommend this movie to anyone who likes to watch funny movies. I am going to give Jackass: The Movie a 7/10.

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: In other words, a movie about nothing. I have never seen such a bad movie. It was so boring that I could not even get through it. I watched the first hour and a half and I still had no idea what was going on. I was just sitting there waiting for something to happen, but nothing did. It was just a bunch of scenes with no connection 

In [32]:
for prompt_type in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (llama_model, llama_tokenizer),
        df=test_sample_df,
        prompt_type=prompt_type,
        top_k=300,
        top_p=0.95,
        temperature=0.7,
        batch_size = 2
    )

100%|██████████| 15/15 [11:09<00:00, 44.65s/it]



--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: I have watched every episode of Jackass and this is one of the funniest films I have ever seen. I can not believe that there were people who did not like it. The best part of the movie was the fight between Steve-O and the bull. The whole scene is hilarious. I recommend this movie to anyone who likes a good laugh. The best part of the film is the fight between Steve-O and the bull. This film is a must see for everyone. I give this film a 10/10. I give this film a 10/10. I give this film a 10/10. I give this film a 10/10. I give this film a 10/10. I give this film a 10/10. I give this film a 10/

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: I can't say anything bad about the cast. As usual, Tommy Lee Jones is a great actor, and he's surrounded by a wonderful supporting cast. But the movie itself is a mess. The screenplay is amateurish, the plot is c

100%|██████████| 15/15 [08:09<00:00, 32.61s/it]


--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: In theaters October 14th, 2002. Jackass: The Movie is a movie of pranks and stunts that will make you laugh, and some stunts will make you cringe. Jackass: The Movie is a hilarious comedy, and will keep you laughing. I gave Jackass: The Movie 7/10. Jackass: The Movie is a good movie. I recommend you see it. If you have a weak stomach or are easily offended, don't see it. Jackass: The Movie is rated R for strong crude and sexual humor, nudity, drug use

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: The plot is muddled and the flashbacks are random and hard to follow. Denzel Washington, as usual, delivers a very good performance as the hard-bitten, hard-drinking, hard-living cop, and his character is interesting. The rest of the cast is fine, with John Goodman and Jill Clayburgh in particular delivering strong performances. The plot is muddled, howeve

##### **VARYING TEMPERATURE**

In [33]:
for prompt_type in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (llama_model, llama_tokenizer),
        df=test_sample_df,
        prompt_type=prompt_type,
        top_k=50,
        top_p=0.95,
        temperature=0.1,
        batch_size = 2
    )

100%|██████████| 15/15 [11:09<00:00, 44.65s/it]



--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: Jackass: The Movie is a movie that is not for the faint of heart. It is a movie that is filled with stunts that are so outrageous that you will be amazed at how they pulled them off. The movie is filled with stunts that are so outrageous that you will be amazed at how they pulled them off. The movie is filled with stunts that are so outrageous that you will be amazed at how they pulled them off. The movie is filled with stunts that are so outrageous that you will be amazed at how they pulled them off. The movie is filled with stunts that are so outrageous that you will be amazed at how they pulled them off. The movie is filled with stunts that are so outrageous that you will be amazed at how they pulled them off. The movie is

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: I'm not sure what the hell this movie was about. I'm not sure what the hell th

100%|██████████| 15/15 [08:09<00:00, 32.62s/it]


--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: The movie is a compilation of the best stunts from the first three Jackass movies. The movie is very funny and entertaining. The movie is very funny and entertaining. The movie is very funny and entertaining. The movie is very funny and entertaining. The movie is very funny and entertaining. The movie is very funny and entertaining. The movie is very funny and entertaining. The movie is very funny and entertaining. The movie is very funny and entertaining. The movie is very funny and entertaining. The movie is very funny and entertaining. The movie is very funny and entertaining

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: The movie is a slow-paced, dark, and depressing movie. The movie is not for everyone. The movie is not for those who like action movies. The movie is not for those who like happy movies. The movie is not for those who like movie

In [ ]:
for prompt_type in ['label-moviename', 'label-moviename-summary']: 
    evaluate_model(
        model_and_tokenizer = (llama_model, llama_tokenizer),
        df=test_sample_df,
        prompt_type=prompt_type,
        top_k=50,
        top_p=0.95,
        temperature=1.3,
        batch_size = 2
    )

100%|██████████| 15/15 [11:09<00:00, 44.66s/it]



--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: This film had me in pee-my-pants gut-bursting laughs for the entire show. I mean, come on, those pratfalls are so nasty! Some of the most original stunts have been compiled here for your viewing pleasure. This movie delivers, even if at times you question the point. The cast for this is as awesome as it gets - Bam! Me! Rory! Seannie! Daddy! Johnny (rest his pee-pee in peace) and all the other lovely cretins are all there to have fun. Sure, most are really crappy stunts, but they look painful as hell. And you have to appreciate the pain in these guys. My favorites would have to be the peep show and the fart. I'm just saying...if you thought "Jackass the show

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: This movie stars Tommy Lee Jones as Dave Robertson who is a retired New Orleans Homicide Detective who has come down from Alaska to track down a mis

100%|██████████| 15/15 [08:08<00:00, 32.60s/it]


--- First 30 Generated Examples ---
1. Movie: Jackass: The Movie
   Sentiment: positive
   Generated Review: (Imdb.com) The movie is great, but the plot is rather lame. Some examples of the scenes in this movie: Bam Margera falls down the street at over 75 MPH, they have a naked chicken fight, they steal a car, Ryan Dunn has to piss on his buddy, all the way up to the end which has a hilarious ending. Overall, this is a good movie for jackass fans. (8.4 out of 10). 8 out of 10 jackass heads- Highly recommended, even if you are not a "jackass fan", you'

2. Movie: In the Electric Mist
   Sentiment: negative
   Generated Review: In the Electric Mist. This is the story. He sees. In the electric mist. You don't see, do you? Yes, this is the story told as far as possible. Someone is killed and the story starts from the first crime. Then comes another, then another, then another. All of them are somehow connected, which is only clear to a certain point, and the detective is convinced everyt